In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/travaux-engages-metropole-lyon.csv", sep=";")
print(df.describe())
display(df)

             numero    code_insee            gid
count  1.319000e+03   1319.000000    1319.000000
mean   2.024778e+08  69258.603487  195480.064443
std    9.287741e+04    121.778735   10836.950822
min    2.018121e+08  69003.000000   74592.000000
25%    2.025078e+08  69157.500000  196242.500000
50%    2.025110e+08  69271.000000  198649.000000
75%    2.025124e+08  69383.000000  199952.500000
max    2.025134e+08  69389.000000  200753.000000


,numero,intervenant,nature_chantier,nature_travaux,etat,date_debut,date_fin,mesures_police,last_update,adresse,commune,code_insee,contact_tel,contact_mail,contact_url,gid
0,202206858,LYON - Direction De La Construction,Travaux sur une parcelle en limite d'espace pu...,Renfort chaussée en vue de la pose d'une grue,Ouvert,2024-06-03,2028-12-31,NaN,2025-02-07 09:02:15.051362+01:00,"10 rue Neyret, 69001 Rue des Tables Claudienn...",Lyon 1er,69381,NaN,NaN,NaN,183025
1,202512731,ORANGE SA,"Branchements réseau < 25 ml TELECOM, RMT ET VIDEO","Intervention sur des chambres, boites, armoire...",Validé,2025-12-24,2026-01-07,"Circulation alternée, Restriction de jour",2025-12-16 15:02:03.127680+01:00,Face au 10 Route du Bruissin (Francheville),Francheville,69089,NaN,NaN,NaN,199942
2,202512882,DCE / RT / Rive Gauche,Émergences et autres équipements d'assainissement,Remise à niveau / entretien des tampons et éme...,Validé,2025-12-15,2026-01-16,NaN,2025-12-04 12:02:37.539147+01:00,"Avenue d'Oschatz, de Rue Gaston Monmousseau a...",Venissieux,69259,04 28 67 55 00,NaN,NaN,199946
3,202512887,ORANGE SA,"Branchements réseau < 25 ml TELECOM, RMT ET VIDEO",Réparation de branchement < 25 ml (télécom ou ...,Validé,2025-12-12,2026-01-19,NaN,2025-12-04 12:02:38.122193+01:00,à côté 3 Chemin de Chantegrillet (Francheville),Francheville,69089,NaN,NaN,NaN,199947
4,202513013,DCE / RT / Rive Droite Sud,"Construction, renouvellement et réparation rés...",Réparation de canalisation d'assainissement,Validé,2025-12-08,2025-12-19,NaN,2025-12-04 12:02:38.969904+01:00,35 Rue Pierre Dupont (Lyon 1er Arrondissement),Lyon 1er,69381,04 28 67 55 20,NaN,NaN,199954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,202512873,RCU Centre Métropole - ELM (Dalkia),"Construction, renouvellement et réparation rés...",Construction de réseau sans branchement (Chauf...,Validé,2025-12-02,2025-12-24,NaN,2025-12-16 15:02:03.235650+01:00,"Du 16 Rue du 8 Mai 1945, au 104 Place du 8 Mai...",Saint Priest,69290,NaN,NaN,NaN,199822
1315,202301053,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-02-07,2026-08-31,NaN,2023-02-07 18:07:40.672980+01:00,- Rue Rue Florian (Villeurbanne) : De Avenue A...,Villeurbanne,69266,NaN,NaN,NaN,135907
1316,202301060,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-01-23,2026-08-31,NaN,2023-02-07 18:07:10.246549+01:00,Tracé T6N du carrefour Zola/Verlaine Bourgey a...,Villeurbanne,69266,NaN,NaN,NaN,135939
1317,202301064,AOMTL (SYTRAL Mobilités),Aménagement pour les transports en commun,Création / aménagement de voie Bus / Tram,Ouvert,2023-01-23,2026-08-31,NaN,2023-02-07 18:07:31.539607+01:00,Traversée futur Parc - Usine ACI Villeurbanne ...,Villeurbanne,69266,NaN,NaN,NaN,135921


### Pretraitement

In [ ]:
import pandas as pd
import json

# --- 1. GÉNÉRATION DU FICHIER POINTS ROUGES (Coordonnées + Description) ---

with open('data_coord/points-rouges-200046977.geojson', 'r') as f:
    geojson_data = json.load(f)

# Extraction des coordonnées (longitude, latitude) et des propriétés
points_rouges_data = []
for feature in geojson_data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']
    
    points_rouges_data.append({
        'longitude': coords[0],
        'latitude': coords[1],
        'commune_insee': props.get('commune'), # 
        'description': props.get('description') # 
    })

df_points_rouges_final = pd.DataFrame(points_rouges_data)

# Sauvegarde du premier fichier
df_points_rouges_final.to_csv('points_rouges_lyon_complet.csv', index=False)


# --- 2. GÉNÉRATION DU FICHIER RÉPONSES (Questionnaire + Lieu) ---

# Chargement du CSV original
df_reponses = pd.read_csv('data_coord/reponses-epci-200046977.csv', low_memory=False, quotechar='"')

# Sélection des colonnes stratégiques selon la notice [cite: 86, 88, 90]
cols_reponses = [
    'insee',        # Lieu (Code INSEE) [cite: 21, 29]
    'q6',           # Fréquence de pratique [cite: 86]
    'q14',          # Sentiment de sécurité [cite: 86]
    'q18',          # Dangerosité carrefours [cite: 88]
    'q21',          # État de l'entretien [cite: 88]
    'q28',          # Conflit stationnement motorisé [cite: 88]
    'q34_texte',    # Les 3 priorités d'amélioration [cite: 88]
    'q35',          # Commentaire libre (NLP) [cite: 88]
    'q38_texte',    # Situations de violence vécues [cite: 90]
    'score'         # Note globale de la commune [cite: 30]
]

# Filtrage et nettoyage
df_reponses_final = df_reponses[cols_reponses].copy()
df_reponses_final['q35'] = df_reponses_final['q35'].str.replace(r'\r+|\n+', ' ', regex=True).fillna('')

# Sauvegarde du second fichier
df_reponses_final.to_csv('reponses_questionnaire_lyon.csv', index=False)

print("Les deux fichiers ont été générés avec succès :")
print("- points_rouges_lyon_complet.csv (Données cartographiques)")
print("- reponses_questionnaire_lyon.csv (Données thématiques et profils)")